Post Train RNN

In [1]:
#pip install darts

In [2]:
#pip install plotting

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
import os
sys.path.append("../") # go to parent dir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

from darts import TimeSeries
from darts.models import FFT, AutoARIMA, ExponentialSmoothing, Theta
from darts.metrics import mae, rmse
from darts.utils.missing_values import fill_missing_values
from darts.utils.statistics import plot_hist

from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    RegressionModel
)

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

ModuleNotFoundError: No module named 'darts'

Load data teste and models

In [ ]:
DT = pd.read_excel('/kaggle/input/dnbcsteste/DNteste.xlsx', index_col=0)
DT.index = pd.to_datetime(DT.index)
DT

In [ ]:
#train DATA
split = 0.5
n = len(DT)
input_df = DT[0:int(n)] #100% 
input_df = input_df.transpose()
input_df = input_df.drop(['pbh','pwh','fq','Zc.1','I','qc','P','f','Zc'])
input_df = input_df[:-1]
input_df = input_df.transpose()
input_df 
input_TS = TimeSeries.from_dataframe(input_df) 
#input_TS['Pin'].plot()
target_train_input, target_val_input = input_TS.split_after(split)


In [ ]:
input_TS['q'].plot()

Separate F and Zc

In [ ]:
f_df = DT[0:int(n)] #100%
f_df = f_df.transpose()
f_df = f_df.drop(['pbh','pwh','fq','Zc.1','I','qc','P','Zc','H','Pin','q'])
f_df = f_df[:-1]
f_df = f_df.transpose()
f_df
f_TS = TimeSeries.from_dataframe(f_df) 
f_TS.plot()
covf_train_input, covf_val_input = f_TS.split_after(split)

In [ ]:
Zc_df = DT[0:int(n)] #100%
Zc_df = Zc_df.transpose()
Zc_df = Zc_df.drop(['pbh','pwh','fq','Zc.1','I','qc','P','f','H','Pin','q'])
Zc_df = Zc_df[:-1]
Zc_df = Zc_df.transpose()
Zc_df
Zc_TS = TimeSeries.from_dataframe(Zc_df) 
Zc_TS.plot()
covZc_train_input, covZc_val_input = Zc_TS.split_after(split)

Load RN trainig models

In [ ]:
model_BRNN = BlockRNNModel.load("/kaggle/input/trainrnn/my_model_brnn_T1.pt")
model_RM = RegressionModel.load("/kaggle/input/trainrnn/my_model_regression.pkl")
model_NB = NBEATSModel.load("/kaggle/input/trainrnn/NBEATSModel_bcs.pt")

In [ ]:
model_NB.fit(
    target_train_input,
    past_covariates = covf_train_input.stack(covZc_train_input),
    verbose = False,
    epochs = 100
)
model_NB.extreme_lags

In [ ]:
def eval_model(model):
    pred_series = model.predict(n=10)
    plt.figure(figsize=(8, 5))
    input_TS.plot(label="actual")
    pred_series.plot(label="forecast")
    plt.title("rmse: {:.2f}%".format(rmse(pred_series, target_val_input)))
    plt.legend()
    return pred_series

In [ ]:
pred_series_NB = eval_model(model_NB)


In [ ]:
pred_series_NB
e1 = rmse(pred_series_NB, target_val_input)
e1

In [ ]:
df_Model_NB = pred_series_NB.pd_dataframe()

df_Target = target_val_input.pd_dataframe()
df_Target 
df_1 = df_Target.iloc[:10,:]
df_2 = df_Target.iloc[10:,:]
#df_1.plot(subplots=True, figsize=(6, 6),x_compat=True)
df_M_1 = pd.merge(df_Model_NB, df_1, on ='time')
df_M_1.plot(subplots=True, figsize=(6, 6),x_compat=True)
df_M_1.rename(columns = {'q_x': 'q_Pred', 'Pin_x': 'Pin_Pred', 'H_x': 'H_Pred', 'q_y': 'q_Val', 'Pin_y': 'Pin_Val', 'H_y': 'H_Val'}, inplace = True)

In [ ]:
#plot each series
plt.plot(df_M_1['q_Pred'], label='q_Pred', color='green')
plt.plot(df_M_1['q_Val'], label='q_Val', color='blue')
# adding title to the plot
plt.title('Vaçao Norm',fontsize=14)
# adding Label to the x-axis
plt.xlabel('Time',fontsize=14)
# adding legend to the curve
plt.legend(title='Legend')
#display plot
plt.show()

In [ ]:
#plot each series
plt.plot(df_M_1['Pin_Pred'], label='Pin_Pred', color='green')
plt.plot(df_M_1['Pin_Val'], label='Pin_Val', color='blue')
# adding title to the plot
plt.title('Pressao Intake Norm',fontsize=14)
# adding Label to the x-axis
plt.xlabel('Time',fontsize=14)
# adding legend to the curve
plt.legend(title='Legend')
#display plot
plt.show()

In [ ]:
#plot each series
plt.plot(df_M_1['H_Pred'], label='H_Pred', color='green')
plt.plot(df_M_1['H_Val'], label='H_Val', color='blue')
# adding title to the plot
plt.title('Head Norm',fontsize=14)
# adding Label to the x-axis
plt.xlabel('Time',fontsize=14)
# adding legend to the curve
plt.legend(title='Legend')
#display plot
plt.show()

In [ ]:
model_BRNN.fit(
    target_train_input,
    val_series = target_val_input,
    verbose = False,
    epochs = 100
)

def eval_model2(model):
    pred_series = model.predict(n=10)
    plt.figure(figsize=(8, 5))
    input_TS.plot(label="actual")
    pred_series.plot(label="forecast")
    plt.title("rmse: {:.2f}%".format(rmse(pred_series, target_val_input)))
    plt.legend()
    return pred_series

In [ ]:
pred_series_BRNN = eval_model2(model_BRNN)
model_BRNN.extreme_lags
e2 = rmse(pred_series_BRNN, target_val_input)
e2

In [ ]:
df_Model_NB = pred_series_BRNN.pd_dataframe()

df_Target = target_val_input.pd_dataframe()
df_Target 
df_1 = df_Target.iloc[:10,:]
df_2 = df_Target.iloc[10:,:]
#df_1.plot(subplots=True, figsize=(6, 6),x_compat=True)
df_M_2 = pd.merge(df_Model_NB, df_1, on ='time')
df_M_2.plot(subplots=True, figsize=(6, 6),x_compat=True)
df_M_2.rename(columns = {'q_x': 'q_Pred', 'Pin_x': 'Pin_Pred', 'H_x': 'H_Pred', 'q_y': 'q_Val', 'Pin_y': 'Pin_Val', 'H_y': 'H_Val'}, inplace = True)

In [ ]:
#plot each series
plt.plot(df_M_2['q_Pred'], label='q_Pred', color='green')
plt.plot(df_M_2['q_Val'], label='q_Val', color='blue')
# adding title to the plot
plt.title('Vaçao Norm',fontsize=14)
# adding Label to the x-axis
plt.xlabel('Time',fontsize=14)
# adding legend to the curve
plt.legend(title='Legend')
#display plot
plt.show()


In [ ]:
#plot each series
plt.plot(df_M_2['Pin_Pred'], label='Pin_Pred', color='green')
plt.plot(df_M_2['Pin_Val'], label='Pin_Val', color='blue')
# adding title to the plot
plt.title('Pressao Intake Norm',fontsize=14)
# adding Label to the x-axis
plt.xlabel('Time',fontsize=14)
# adding legend to the curve
plt.legend(title='Legend')
#display plot
plt.show()

In [ ]:
#plot each series
plt.plot(df_M_2['H_Pred'], label='H_Pred', color='green')
plt.plot(df_M_2['H_Val'], label='H_Val', color='blue')
# adding title to the plot
plt.title('Head Norm',fontsize=14)
# adding Label to the x-axis
plt.xlabel('Time',fontsize=14)
# adding legend to the curve
plt.legend(title='Legend')
#display plot
plt.show()

Fit models

In [ ]:
eval_model(model_loaded2)

In [ ]:
model_loaded1.fit(
    target_train_input,
    past_covariates = covf_train_input.stack(covZc_train_input),
)
model_loaded1.extreme_lags

In [ ]:
def eval_model4(model):
    pred_series = model.predict(n=1)
    plt.figure(figsize=(8, 5))
    input_TS.plot(label="actual")
    pred_series.plot(label="forecast")
    plt.title("rmse: {:.2f}%".format(rmse(pred_series, target_val_input)))
    plt.legend()
    plt.savefig('Reg_Model.png')
    return pred_series
    


In [ ]:
df_Model_RM = eval_model4(model_RM)
model_RM.extreme_lags

In [ ]:
df_Model_RM = pred_series_NB.pd_dataframe()

df_Target = target_val_input.pd_dataframe()
df_Target 
df_1 = df_Target.iloc[:1,:]
df_2 = df_Target.iloc[1:,:]
#df_1.plot(subplots=True, figsize=(6, 6),x_compat=True)
df_M_3 = pd.merge(df_Model_RM, df_1, on ='time')
df_M_3.plot(subplots=True, figsize=(6, 6),x_compat=True)
df_M_3.rename(columns = {'q_x': 'q_Pred', 'Pin_x': 'Pin_Pred', 'H_x': 'H_Pred', 'q_y': 'q_Val', 'Pin_y': 'Pin_Val', 'H_y': 'H_Val'}, inplace = True)

In [ ]:
def HF_model(model, past_covariates=None, future_covariates=None):
    # Past and future covariates are optional because they won't always be used in our tests
    
    # We backtest the model on the last 20% of the flow series, with a horizon of 10 steps:
    backtest = model.historical_forecasts(series=flow, 
                                          past_covariates=past_covariates,
                                          future_covariates=future_covariates,
                                          start=0.8, 
                                          retrain=False,
                                          verbose=True, 
                                          forecast_horizon=10)
    
    flow[-len(backtest)-100:].plot()
    backtest.plot(label='backtest (n=10)')
    print('Backtest RMSE = {}'.format(rmse(flow, backtest)))